In [ ]:
from __future__ import division
%pylab inline
import numpy as np
import scipy.spatial.distance as dist
from scipy import stats
import pickle
import math
import collections
import numpy as np

In [ ]:
def readRough(name="traininglabels"):
    with open(name) as label:
        labels = []
        for line in label:
            labels.append(line[0])
    return labels

In [ ]:
label = readRough()
print len(label)
class_ = sorted(collections.Counter(label).items())
print class_

In [ ]:
with open('labelstats.txt', 'wb') as fp:
    pickle.dump(class_, fp)

In [ ]:
with open ('labelstats.txt', 'rb') as fp:
    class_ = pickle.load(fp)

In [ ]:
print class_

In [ ]:
def inputDigit(name="trainingimages"):
    with open(name) as digitTxt:
        image = [list(line)[0:28] for line in digitTxt]
        rows = len(image)
    return image


In [ ]:
data_training = inputDigit()

In [12]:
# " " = high   =  1
# "%" = low    =  0
# len(row) = 10
def countNum(name = "yes_train.txt"):
    with open(name) as dataset:
        num_lines = sum(1 for line in dataset)
        return num_lines / 28
    


for training 

In [13]:
# label yes = 1
# label no = 0
Num_sample_yes =  countNum()
Num_sample_no =  countNum("no_train.txt")
print Num_sample_yes,Num_sample_no

140 131


In [57]:
Num_test_yes =  countNum("yes_test.txt")
Num_test_no =  countNum("no_test.txt")
print Num_test_yes,Num_test_no

50 50


In [35]:
def inputDigit(name="yes_train.txt"):
    with open(name) as digitTxt:
        image = [list(line)[0:10] for line in digitTxt]
    return image

In [37]:
data_yes = inputDigit()
data_no = inputDigit("no_train.txt")
test_yes = inputDigit("yes_test.txt")
test_no = inputDigit("no_test.txt")

In [90]:
#   yes is class 0
#   no is class 1
k = 10
    
def training(yes_train, no_train, yes_num, no_num):
    # " " = high   =  1
    # "%" = low    =  0
    
    class_total = [yes_num, no_num]

    training0 = np.zeros(shape=(2, 25, 10))   #high
    training1 = np.zeros(shape=(2, 25, 10))   #low

    count = 0
    for i in range(yes_num):
        for row in range(25):
            for col in range(10):
                if yes_train[i*28 + row][col] == ' ':
                    training1[0][row][col] += 0
                    training0[0][row][col] += 1
                else:
                    training1[0][row][col] += 1
                    training0[0][row][col] += 0
                    
    for i in range(no_num):
        for row in range(25):
            for col in range(10):
                if no_train[i*28 + row][col] == ' ':
                    training1[1][row][col] += 0
                    training0[1][row][col] += 1
                else:
                    training1[1][row][col] += 1
                    training0[1][row][col] += 0

    for i in range(2):
        training1[i] = (training1[i] + k) / (class_total[i] + k * 2)
        training0[i] = (training0[i] + k) / (class_total[i] + k * 2)
#     print(training0[1][0])
#     print(training1[1][0])
    
    return training0, training1

In [91]:
training0, training1 = training(data_yes, data_no, Num_sample_yes, Num_sample_no)

In [92]:
# log P(class) + log P(f1,1 | class) + log P(f1,2 | class) + ... + log P(f28,28 | class)

# test_rough = inputDigit(name = "digitdata/testimages")
def testing(yes_test, no_test, Num_test_yes, Num_test_no):

    data_test = np.concatenate( (yes_test, no_test) , axis = 0 )
# #     print len(data_test) / 28
#     return 

    total = Num_test_yes + Num_test_no
    answer = np.zeros(total)
    
    class_total = [Num_test_yes, Num_test_no]
    
    for i in range(total):
        test_image = np.zeros(shape=(25,10))
        for row in range(25):
            for col in range(10):
                if data_test[i*28+row][col] == ' ':
                    test_image[row][col] = 1
                else:
                    test_image[row][col] = 0
                    

        posteriori = np.zeros(2)
        for class_num in range(2):
            posteriori[class_num] = math.log(class_total[class_num])
            for row in range(25):
                for col in range(10):
                    if test_image[row][col] == 0:
                        posteriori[class_num] += math.log(training0[class_num][row][col])
                    else:
                        posteriori[class_num] += math.log(training1[class_num][row][col] )   
        answer[i] = np.argmax(posteriori)

    print(answer)



In [93]:
testing(test_yes, test_no, Num_test_yes, Num_test_no)

[ 1.  1.  0.  0.  0.  1.  0.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  0.  1.  1.  1.  1.  0.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.  1.
  1.  1.  0.  1.  0.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.  0.  1.  0.
  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


In [ ]:
print()

In [33]:

# def preprossesData(Num_sample, name="yes_train.txt"):
#     with open(name) as rowdata:
#         dataset = np.zeros((Num_sample,250))
#         row = 0
#         ele_count = 0
#         for line in rowdata:
#             for i in range(10):
#                 if ele_count == 249:
#                     ele_count = 0
#                 ele_count += 1
#                 if line[i] == " ":
#                     dataset[row][ele_count] = 1
#                 elif line[i] == "%":
#                     dataset[row][ele_count] = 0
            
#             row += 1
            
#     return image


In [32]:
# print preprossesData(Num_sample_yes,"yes_train.txt")